In [28]:
import numpy as np
import pandas as pd
from sklearn import *

C:\Users\luco\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\luco\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\luco\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 an

In [41]:
df = pd.read_csv('letter-recognition.data', header=None)
df['target'] = df[0].apply(lambda x: ord(x) - ord('A'))
del df[0]
print df.dtypes
print df.head()

1         int64
2         int64
3         int64
4         int64
5         int64
6         int64
7         int64
8         int64
9         int64
10        int64
11        int64
12        int64
13        int64
14        int64
15        int64
16        int64
target    int64
dtype: object
   1   2  3  4  5   6   7  8  9  10  11  12  13  14  15  16  target
0  2   8  3  5  1   8  13  0  6   6  10   8   0   8   0   8      19
1  5  12  3  7  2  10   5  5  4  13   3   9   2   8   4  10       8
2  4  11  6  8  6  10   6  2  6  10   3   7   3   7   3   9       3
3  7  11  6  6  3   5   9  4  6   4   4  10   6  10   2   8      13
4  2   1  3  1  1   8   6  6  6   6   5   9   1   7   5  10       6


In [55]:
data = df.values
np.random.shuffle(data)
r, c = data.shape
training_split = 0.7
training_rows = int(training_split * r)
training_data = data[:training_rows, :]
test_data = data[training_rows:, :]
print training_data.shape
print test_data.shape

(14000L, 17L)
(6000L, 17L)


In [58]:
train_x = training_data[:, :-1]
train_y = training_data[:, -1]
test_x = test_data[:, :-1]
test_y = test_data[:, -1]

In [59]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_x, train_y)